In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

C:\Users\pravi\anaconda3\envs\deepl2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cpu')

In [4]:
num_epochs = 15
batch_size = 4
learning_rate =0.001

In [5]:
transform = transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [6]:
train_dataset = torchvision.datasets.CIFAR10(root='data',
                                            train=True,
                                            download=True,
                                            transform=transform)

Files already downloaded and verified


In [7]:
test_dataset = torchvision.datasets.CIFAR10(root='data',
                                            train=False,
                                            download=True,
                                            transform=transform)

Files already downloaded and verified


In [8]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [9]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [10]:
examples = iter(train_loader)
samples ,labels = examples.next()
samples.shape

torch.Size([4, 3, 32, 32])

In [11]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)

In [12]:
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [13]:
classes

('plane',
 'car',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck')

In [14]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [15]:
model = ConvNet().to(device)

In [16]:
model

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [18]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        #forward
        outputs = model(images)
        loss = criterion(outputs,labels)
        #backword
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}],step [{i+1}/{n_total_steps}],loss: {loss.item():.4f}')
print("Finished Traing")

Epoch [1/15],step [2000/12500],loss: 2.3207
Epoch [1/15],step [4000/12500],loss: 2.2885
Epoch [1/15],step [6000/12500],loss: 2.2923
Epoch [1/15],step [8000/12500],loss: 2.3120
Epoch [1/15],step [10000/12500],loss: 2.2849
Epoch [1/15],step [12000/12500],loss: 2.3432
Epoch [2/15],step [2000/12500],loss: 2.1989
Epoch [2/15],step [4000/12500],loss: 2.5534
Epoch [2/15],step [6000/12500],loss: 2.1769
Epoch [2/15],step [8000/12500],loss: 1.8622
Epoch [2/15],step [10000/12500],loss: 2.3364
Epoch [2/15],step [12000/12500],loss: 2.4359
Epoch [3/15],step [2000/12500],loss: 1.8720
Epoch [3/15],step [4000/12500],loss: 1.8214
Epoch [3/15],step [6000/12500],loss: 1.3041
Epoch [3/15],step [8000/12500],loss: 1.5110
Epoch [3/15],step [10000/12500],loss: 1.8302
Epoch [3/15],step [12000/12500],loss: 1.1715
Epoch [4/15],step [2000/12500],loss: 2.1652
Epoch [4/15],step [4000/12500],loss: 1.8550
Epoch [4/15],step [6000/12500],loss: 2.4656
Epoch [4/15],step [8000/12500],loss: 1.3804
Epoch [4/15],step [10000/1

In [19]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs,1)
        n_samples += labels.size(0)
        n_correct += (predicted==labels).sum().item()
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label==pred):
                n_class_correct[label] +=1
            n_class_samples[label]+=1
    acc = 100.0 * n_correct/n_samples
    print(f'accuracy: {acc}%')
    for i in range(10):
        acc =100.0 *n_class_correct[i]/n_class_samples[i]
        print(f'accuracy of {classes[i]}:{acc}%')

accuracy: 61.64%
accuracy of plane:67.7%
accuracy of car:72.8%
accuracy of bird:39.6%
accuracy of cat:34.7%
accuracy of deer:55.2%
accuracy of dog:53.4%
accuracy of frog:80.1%
accuracy of horse:66.5%
accuracy of ship:75.0%
accuracy of truck:71.4%


In [20]:
batch_size

4